In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
# https://matplotlib.org/users/customizing.html
# print(plt.style.available) # uncomment to print all styles
import seaborn as sns
sns.set(font_scale=2)
plt.style.use('seaborn-whitegrid')
mpl.rcParams['figure.figsize'] = (10.0, 8.0)

# 1) Density of Floating Point Numbers

The following code snippet generates all possible floating point numbers in a floating point system and shows them in a plot to illustrate their density.

In [3]:
significand_bits = 2
exponent_min = -4
exponent_max = 4

fp_numbers = []
for exp in range(exponent_min, exponent_max+1):
    for sbits in range(0, 2**significand_bits):
        significand = 1 + sbits/2**significand_bits 
        fp_numbers.append(significand * 2**exp)
        
fp_numbers = np.array(fp_numbers)
fp_numbers

In [4]:
plt.figure(figsize=(10,1))
plt.plot(fp_numbers, np.ones_like(fp_numbers), "o");

In [5]:
plt.figure(figsize=(10,1))
plt.plot(fp_numbers, np.ones_like(fp_numbers), "o");
plt.axis([0,1.6, 0,2])

# 2) Picking apart a floating point number (IEEE double precision)

In [6]:
# Never mind the details of this function...

def pretty_print_fp(x):
    print("---------------------------------------------")
    print("Floating point structure for %r" % x)
    print("---------------------------------------------")
    import struct
    s = struct.pack("d", x)

    def get_bit(i):
        byte_nr, bit_nr = divmod(i, 8)
        return int(bool(
            s[byte_nr] & (1 << bit_nr)
            ))

    def get_bits(lsb, count):
        return sum(get_bit(i+lsb)*2**i for i in range(count))

    # https://en.wikipedia.org/wiki/Double_precision_floating-point_format

    print("Sign bit (1:negative):", get_bit(63))
    
    stored_exponent = get_bits(52, 11)
    fraction = get_bits(0, 52)
    
    if stored_exponent == 0:
        exponent = -1022
        significand = fraction
    else:
        exponent = stored_exponent - 1023
        significand = fraction + 2**52
 
    print("Mathematical exponent (decimal): m = %d" % exponent)
    print("Shifted exponent value (decimal): c = %d" % stored_exponent)
    print("Stored exponent value (bin): c = {0:011b}".format(stored_exponent) )

    print("Fractional part of significand (bin): f = {0:052b}".format(fraction))


In [7]:
pretty_print_fp(float(1.0))

Things to try:

* Twiddle the sign bit
* 1,2,4,8
* 0.5,0.25
* $2^{\pm 1023}$, $2^{\pm 1024}$
* `float("inf")`, `float("-inf")`
* `float("nan")`
* $2^{\pm 1040}$
* `float("+0")`, `float("-0")`

In [8]:
bin(1023)

In [9]:
pretty_print_fp(float(0.1))

# 3) Checking the range of integers

We discussed in class that we can obtain the range of integers that can be represented exactly as

$$[1,2^{p}]$$

So let's try this using Python (where double precision gives p=53)

In [ ]:
# This number is within the range
2**52

In [ ]:
# We add one to get the next integer 
2**52 + 1

In [ ]:
# All good so far, right? Let's check the integer:
2**53

In [ ]:
# If I add one, what do you expect us to get?
type(2**53 + 1)

Humm... not what you expected right? Why do you think this happened?

In [ ]:
print(2**53)
print(2**53 + 1.0)
print(2**53 + 2.0)
print(2**53 + 3.0)
print(2**53 + 4.0)

### Integer type

In [ ]:
# to find the maximum integer (using 64-bit signed integer)
# uses only 63 bits, since one is reserved for the sign
maxint = 0
for i in range(63):
     maxint += 2**i
maxint

# 4) What will happen when we run these code snippets?

A) it won't stop (infinite loop)

B) it will stop when b reaches underflow

C) it will stop when b reaches machine epsilon

D) none of the above

In [ ]:
a = 1.0
while a > 0.0: 
    a = a / 2.0
    print(a)
    #print("% .16e"% (a)) 

A) it won't stop (infinite loop)

B) it will stop when b reaches underflow

C) it will stop when b reaches machine epsilon

D) none of the above

In [ ]:
a = 1.0
while a != np.inf:
    a = a * 2.0
    print(a)
    #print("% .16e"% (a)) 

Let's write the following number (not exacly the definition of OFL, but close)

In [ ]:
float(2**1023)

In [ ]:
# Try the UFL definition
2**(1023+1)*(1-2**(-53))

# Can we make this work?